In [25]:
import torch
print(torch.__version__)
print (torch.cuda.is_available())
print (torch.cuda.device_count())

1.13.1+cu117
True
3


This notebook implements a character-level transformer model. The goal is to replicate a lecture to learn how to make a transformer architecture.

This is a decoder-only architecture. There is no sequence-to-sequence functionality, no clasification of text, other NLP tasks... just text generation / pure language modeling.  

Source:
https://www.youtube.com/watch?v=kCc8FmEb1nY

# Dataset loading & exploration

In [26]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# !mkdir ../data/tinyshakespeare -p
# !mv input.txt ../data/tinyshakespeare/input.txt

In [27]:
# Observe dataset size, first 1000 characters
with open ('../data/tinyshakespeare/input.txt', 'r') as f:
    text = f.read()

print ("Length: ", len(text))
print (text[:1000])

Length:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thir

In [28]:
# Print sorted vocabulrary and vocabulary size
vocab = sorted(set(text))
print ("Vocabulary: ", vocab)
print ("Vocabulary size: ", len(vocab))

Vocabulary:  ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Vocabulary size:  65


# Tokenization, train-test split

In [29]:
# Create tokenizer to encode and decode text by mapping vocabulary to integers
stoi = {c: i for i, c in enumerate(vocab)}
itos = {i: c for i, c in enumerate(vocab)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[c] for c in l])

print(encode(text[:1000]))

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43, 43, 42, 1, 39, 52, 63, 1, 44, 59, 56, 58, 46, 43, 56, 6, 1, 46, 43, 39, 56, 1, 51, 43, 1, 57, 54, 43, 39, 49, 8, 0, 0, 13, 50, 50, 10, 0, 31, 54, 43, 39, 49, 6, 1, 57, 54, 43, 39, 49, 8, 0, 0, 18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 37, 53, 59, 1, 39, 56, 43, 1, 39, 50, 50, 1, 56, 43, 57, 53, 50, 60, 43, 42, 1, 56, 39, 58, 46, 43, 56, 1, 58, 53, 1, 42, 47, 43, 1, 58, 46, 39, 52, 1, 58, 53, 1, 44, 39, 51, 47, 57, 46, 12, 0, 0, 13, 50, 50, 10, 0, 30, 43, 57, 53, 50, 60, 43, 42, 8, 1, 56, 43, 57, 53, 50, 60, 43, 42, 8, 0, 0, 18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 18, 47, 56, 57, 58, 6, 1, 63, 53, 59, 1, 49, 52, 53, 61, 1, 15, 39, 47, 59, 57, 1, 25, 39, 56, 41, 47, 59, 57, 1, 47, 57, 1, 41, 46, 47, 43, 44, 1, 43, 52, 43, 51, 63, 1, 58, 53, 1, 58, 46, 43, 1, 54, 43, 53, 54, 50, 43, 8, 0, 0, 13, 50, 50, 10, 0, 35, 43, 1, 49, 52, 53, 61, 5, 

In [30]:
# split dataset into train and validation
cutoff = int(0.9 * len(text))
train_text = text[:cutoff]
val_text = text[cutoff:]

In [31]:
data = torch.tensor(encode(text), dtype=torch.long)
train_data = data[: cutoff]
val_data = data[cutoff:]
(len(train_data), len(val_data))

(1003854, 111540)

# Loading examples, batches, and creating an oracle

In [32]:
# Show the first input sequence and the expected predictions
block_size = 8
example = encode(train_text[:block_size + 1])
example = torch.tensor(example, dtype=torch.long)
print(example)

for idx in range(block_size):
    print(f'For value {example[idx]} the target is {example[idx+1]}')

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
For value 18 the target is 47
For value 47 the target is 56
For value 56 the target is 57
For value 57 the target is 58
For value 58 the target is 1
For value 1 the target is 15
For value 15 the target is 47
For value 47 the target is 58


In [40]:
torch.manual_seed(1337)
batch_size = 4

def get_batch(is_train: bool = True):
    data = train_data if is_train else val_data
    ix = torch.randint(0, len(data)-block_size, (batch_size,)) # should this be len(data) - 1? we need to account for yb
    xb = torch.stack([data[i: i+block_size] for i in ix])
    yb = torch.stack([data[i+1: i+1+block_size] for i in ix])
    return xb, yb

xb, yb = get_batch()

for b in range(batch_size):
    for t in range(block_size):
        print(f"When the input is {xb[b][:t+1].tolist()} the target is {yb[b][t]}")

When the input is [24] the target is 43
When the input is [24, 43] the target is 58
When the input is [24, 43, 58] the target is 5
When the input is [24, 43, 58, 5] the target is 57
When the input is [24, 43, 58, 5, 57] the target is 1
When the input is [24, 43, 58, 5, 57, 1] the target is 46
When the input is [24, 43, 58, 5, 57, 1, 46] the target is 43
When the input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
When the input is [44] the target is 53
When the input is [44, 53] the target is 56
When the input is [44, 53, 56] the target is 1
When the input is [44, 53, 56, 1] the target is 58
When the input is [44, 53, 56, 1, 58] the target is 46
When the input is [44, 53, 56, 1, 58, 46] the target is 39
When the input is [44, 53, 56, 1, 58, 46, 39] the target is 58
When the input is [44, 53, 56, 1, 58, 46, 39, 58] the target is 1
When the input is [52] the target is 58
When the input is [52, 58] the target is 1
When the input is [52, 58, 1] the target is 58
When the input is [52, 

# Bigram model

In [69]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, x, y= None):
        if y is None:
            loss = None
            pred = self.embed(x)

        else:
            pred = self.embed(x)
            loss = F.cross_entropy(pred.view(-1, self.vocab_size), y.view(-1)) # must convert shapes from (B, T, C) to (B*T, C) channels must be second!
        return pred, loss

    def generate(self, x, max_tokens):
        for _ in range(max_tokens):
            logits = self(x)[0] 
            last = logits[:, -1, :] # get prediction for latest token only
            probs = F.softmax(last, dim=-1)  # normalize logits to prob distr
            val = torch.multinomial(probs, num_samples=1) #sample from distr
            x = torch.cat([x,val], dim=1) # add sampled char to input
        return x


m = BigramModel(len(vocab))
print(m(xb, yb)[0].shape, m(xb,yb)[1])

test = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(test, 100)[0].tolist()))


torch.Size([4, 8, 65]) tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [70]:
optimizer = torch.optim.AdamW(m.parameters(), 1e-2)

In [ ]:
batch_size = 32

for epoch in range(100):
    xb, yb = get_batch()
    logits, loss = m(xb, yb)
    loss.backward()
    optimizer.step()
